<a href="https://colab.research.google.com/github/Natty093/hack4her/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#librerias
from google.colab import drive
import pandas as pd
import glob
import locale

try:
    drive.mount('/content/drive')
    print("Google Drive montado correctamente.")
except Exception as e:
    print(f"Error al montar Google Drive: {e}")

# Leer y concatenar CSVs con manejo de codificación
ruta_carpeta = '/content/drive/MyDrive/2022/*.csv'
archivos_csv = glob.glob(ruta_carpeta)

#primero checa si no hay archivos
if not archivos_csv:
    print(f"No se encontraron archivos CSV en la ruta: {ruta_carpeta}")
    df_combinado = pd.DataFrame()
# si paso esto, se crea una lista vacia y se agregan posibles codificaciones por si el archivo no se puede leer bien
else:
    lista_df = []
    codificaciones_a_probar = ['utf-8', 'latin-1', 'ISO-8859-1']

# se leen primero todos los archivos que sean csv, para despues
# probar con las diferentes codificaciones para leerlas
    print(f"\nLeyendo archivos CSV de la ruta: {ruta_carpeta}")
    for archivo in archivos_csv:
        df = None
        for codificacion in codificaciones_a_probar:
            try:
                df = pd.read_csv(archivo, encoding=codificacion)
                print(f"Archivo {archivo} leído con codificación: {codificacion}")
                lista_df.append(df)
                break
            except Exception:
                pass
        if df is None:
            print(f"No se pudo leer el archivo {archivo} con las codificaciones probadas.")
# ya leyó todo, así que los datos los acomodo en un data frame
    df_combinado = pd.concat(lista_df, ignore_index=True) if lista_df else pd.DataFrame()
    print("\nDataFrame combinado creado." if not df_combinado.empty else "\nDataFrame combinado vacío.")

# prueba de que df_combinado tiene datos
if df_combinado.empty:
    print("El DataFrame combinado está vacío. No se puede continuar.")
else:
    # convertir de str a numeros y si no hay nada, se deja como NaN
    for col in ['Venta USD', 'Venta Cajas']:
        if col in df_combinado.columns:
            df_combinado[col] = pd.to_numeric(df_combinado[col], errors='coerce')
            print(f"Columna '{col}' convertida a numérica.")
        else:
            print(f"Advertencia: Columna '{col}' no encontrada.")

    # para la columna de fecha_transaccion se mapean los meses
    if 'Año' in df_combinado.columns and 'Mes' in df_combinado.columns:
        try:
            meses_es = {
                'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4,
                'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8,
                'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
            }
            # proceso de mes a numero
            df_combinado['Mes_num'] = df_combinado['Mes'].astype(str).map(meses_es)
            if df_combinado['Mes_num'].isnull().any():
                print("Advertencia: Hay meses no reconocidos en la columna 'Mes'. Asegúrate que los nombres de los meses en tus CSVs coinciden con el diccionario.")
            # ahora se combinan el año y mes para la feha_transaccion
            df_combinado['fecha_transaccion'] = pd.to_datetime(
                df_combinado['Año'].astype(str) + '-' +
                df_combinado['Mes_num'].astype(str) + '-01',
                format='%Y-%m-%d'
            )
            # se imprime todo
            print("Columna 'fecha_transaccion' creada correctamente.")
            df_combinado.drop(columns=['Mes_num'], inplace=True)

        except Exception as e:
            print(f"Error al crear la columna 'fecha_transaccion': {e}")
    else:
        print("Faltan las columnas 'Año' o 'Mes', no se puede crear 'fecha_transaccion'.")

    # se crea una lista vacía para las características por cliente
    caracteristicas_cliente = []

    # de la database se recolectan datos del cliente (como el ID y tamaño)
    if 'ID Cliente' in df_combinado.columns and 'Tamaño de Cliente' in df_combinado.columns:
        df_clientes = df_combinado[['ID Cliente', 'Tamaño de Cliente']].drop_duplicates(subset=['ID Cliente']).reset_index(drop=True)
        caracteristicas_cliente.append(df_clientes)
        print("DataFrame base de clientes creado.")
    else:
        print("Faltan columnas 'ID Cliente' o 'Tamaño de Cliente'. No se pueden calcular características por cliente.")

    # recencia y frecuencia
    if 'fecha_transaccion' in df_combinado.columns and 'ID Cliente' in df_combinado.columns:
        fecha_ref = df_combinado['fecha_transaccion'].max()
        recencia_df = df_combinado.groupby('ID Cliente')['fecha_transaccion'].max().reset_index(name='ultima_compra')
        recencia_df['recencia_dias'] = (fecha_ref - recencia_df['ultima_compra']).dt.days
        caracteristicas_cliente.append(recencia_df[['ID Cliente', 'recencia_dias']])

        frecuencia_df = df_combinado.groupby('ID Cliente')['fecha_transaccion'].count().reset_index(name='frecuencia_compras')
        caracteristicas_cliente.append(frecuencia_df)

        print("Características 'recencia_dias' y 'frecuencia_compras' calculadas.")
    else:
        print("No se pudo calcular recencia ni frecuencia, faltan columnas necesarias.")

    # valor monetario
    if 'Venta USD' in df_combinado.columns and 'ID Cliente' in df_combinado.columns:
        valor_monetario_df = df_combinado.groupby('ID Cliente')['Venta USD'].sum().reset_index(name='valor_monetario_total')
        caracteristicas_cliente.append(valor_monetario_df)
        print("Característica 'valor_monetario_total' calculada.")
    else:
        print("No se pudo calcular valor monetario, faltan columnas.")

    # categoría preferida
    if 'Categoria' in df_combinado.columns and 'ID Cliente' in df_combinado.columns:
        categoria_preferida_df = df_combinado.groupby('ID Cliente')['Categoria'] \
            .agg(lambda x: x.mode()[0] if not x.mode().empty else 'Desconocido') \
            .reset_index(name='categoria_preferida')
        caracteristicas_cliente.append(categoria_preferida_df)
        print("Característica 'categoria_preferida' calculada.")
    else:
        print("No se pudo calcular categoría preferida, faltan columnas.")

    # variabilidad en venta USD (std)
    if 'Venta USD' in df_combinado.columns and 'ID Cliente' in df_combinado.columns:
        std_venta_df = df_combinado.groupby('ID Cliente')['Venta USD'].std().reset_index(name='std_venta_usd')
        std_venta_df['std_venta_usd'] = std_venta_df['std_venta_usd'].fillna(0) # Rellenar NaN si solo hay una compra
        caracteristicas_cliente.append(std_venta_df)
        print("Característica 'std_venta_usd' calculada.")
    else:
        print("No se pudo calcular variabilidad venta USD, faltan columnas.")

    # === AÑADIR ESTO: Calcular Ventas por Categoría como columnas separadas ===
    if 'Categoria' in df_combinado.columns and 'Venta USD' in df_combinado.columns and 'ID Cliente' in df_combinado.columns:
        ventas_por_categoria_pivot = df_combinado.groupby(['ID Cliente', 'Categoria'])['Venta USD'].sum().unstack(fill_value=0)
        # Asegurarse de que los nombres de las columnas sean válidos y únicos
        ventas_por_categoria_pivot.columns = ['Ventas_' + str(col).replace(' ', '_').replace('-', '_') for col in ventas_por_categoria_pivot.columns]
        ventas_por_categoria_pivot = ventas_por_categoria_pivot.reset_index()
        caracteristicas_cliente.append(ventas_por_categoria_pivot)
        print("Características de ventas por categoría (columnas separadas) calculadas.")
    else:
        print("Advertencia: No se pudieron calcular ventas por categoría como columnas. Faltan columnas 'Categoria', 'Venta USD' o 'ID Cliente'.")

    # todas las características recolectadas se unen en un solo DataFrame por cliente
    if caracteristicas_cliente:
        # inicismos con el primer DataFrame de la lista para luego ir uniendo las demás características, usando el 'ID Cliente' como llave
        df_final = caracteristicas_cliente[0].copy()
        for i, df_feat in enumerate(caracteristicas_cliente[1:]):
            df_final = df_final.merge(df_feat, on='ID Cliente', how='left')

        print("\nDataFrame final con características por cliente creado:")
        print(df_final.head())
        print("\nInformación del DataFrame final:")
        df_final.info()
    else:
        print("No se calcularon características para combinar. df_final está vacío.")
        df_final = pd.DataFrame() # Asegurarse de que df_final existe aunque esté vacío

# RESUMEN FUNAL DE LOS DATOS SE IMPRIMEN PARA ASEGURARSE DE QUE TODO ESTA BIEN S
if not df_final.empty:
    print("\n=== Datos combinados por cliente (Primeras 10 filas) ===")
    print(df_final.head(10))

    print("\n=== Estadísticas resumen del DataFrame final ===")
    print(df_final.describe(include='all'))

    print("\n=== Información del DataFrame final ===")
    print(df_final.info())

    if 'recencia_dias' in df_final.columns:
        print("\n=== Recencia mínima y máxima (días) ===")
        print(f"Min: {df_final['recencia_dias'].min()}, Max: {df_final['recencia_dias'].max()}")

    if 'frecuencia_compras' in df_final.columns:
        print("\n=== Frecuencia mínima y máxima ===")
        print(f"Min: {df_final['frecuencia_compras'].min()}, Max: {df_final['frecuencia_compras'].max()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado correctamente.
No se encontraron archivos CSV en la ruta: /content/drive/MyDrive/2022/*.csv
El DataFrame combinado está vacío. No se puede continuar.

=== Datos combinados por cliente (Primeras 10 filas) ===
   ID Cliente Tamaño de Cliente  recencia_dias  frecuencia_compras  \
0           1             MICRO            303                 115   
1          10            GRANDE            395                 376   
2         100           MEDIANO              0                 616   
3        1001             MICRO              0                 345   
4        1002           MEDIANO              0                 608   
5        1003             MICRO              0                 327   
6        1004             MICRO              0                 470   
7        1005             MICRO              0                 281   
8        100

In [12]:
import numpy as np
import pandas as pd # Asegúrate de importar pandas si no lo hiciste en esta celda
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Variables globales para almacenar los IDs de clientes
# Serán accesibles en la siguiente celda si se ejecuta en el mismo entorno (Colab)
clientes_alto_valor_ids = []
clientes_bajo_valor_ids = []

# Asegúrate de que df_final ha sido creado por la celda anterior
if 'df_final' in locals() and not df_final.empty:
    if 'valor_monetario_total' in df_final.columns:
        # Calcular la mediana del valor monetario total
        mediana_valor = df_final['valor_monetario_total'].median()

        # Crear la nueva columna objetivo
        df_final['EsClienteAltoValor'] = (df_final['valor_monetario_total'] > mediana_valor).astype(int)

        print("\nColumna 'EsClienteAltoValor' creada basada en la mediana del valor monetario.")
        print("Conteo de valores en la nueva columna objetivo:")
        print(df_final['EsClienteAltoValor'].value_counts())

        columna_objetivo = 'EsClienteAltoValor'

        # Usamos una copia para no modificar df_final original en el proceso de entrenamiento
        df_ml_prep = df_final.copy()

        # Guardamos los ID de cliente antes de cualquier transformación para poder unir predicciones después
        clientes_ids_para_prediccion = df_ml_prep['ID Cliente']

        # Columnas a eliminar del conjunto de características (X)
        columnas_a_eliminar_de_X = ['valor_monetario_total', 'ID Cliente', 'ultima_compra', 'EsClienteAltoValor']

        X_ml = df_ml_prep.drop(columns=columnas_a_eliminar_de_X, errors='ignore')
        y_ml = df_ml_prep[columna_objetivo]

        # Revisar y codificar columnas categóricas en X_ml
        cat_cols_ml = X_ml.select_dtypes(include=['object', 'category']).columns.tolist()
        label_encoders_ml = {}
        for col in cat_cols_ml:
            X_ml[col] = X_ml[col].astype(str)
            le = LabelEncoder()
            X_ml[col] = le.fit_transform(X_ml[col])
            label_encoders_ml[col] = le

        for col in X_ml.select_dtypes(include=np.number).columns:
            if X_ml[col].isnull().any():
                X_ml[col] = X_ml[col].fillna(X_ml[col].median())

        if not X_ml.empty and X_ml.select_dtypes(exclude=np.number).empty:
            print(f"\nPreparando datos para predecir: '{columna_objetivo}'")
            print("Variables predictoras (X_ml):", X_ml.columns.tolist())

            X_train, X_test, y_train, y_test = train_test_split(X_ml, y_ml, test_size=0.2, random_state=42)

            print("\nEntrenando el modelo RandomForestClassifier...")
            modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
            modelo_rf.fit(X_train, y_train)
            print("Entrenamiento completado.")


            X_full_for_pred = df_ml_prep.drop(columns=columnas_a_eliminar_de_X, errors='ignore')

            for col in cat_cols_ml:
                if col in X_full_for_pred.columns:
                    X_full_for_pred[col] = label_encoders_ml[col].transform(X_full_for_pred[col].astype(str))

            for col in X_full_for_pred.select_dtypes(include=np.number).columns:
                if X_full_for_pred[col].isnull().any():
                    X_full_for_pred[col] = X_full_for_pred[col].fillna(X_ml[col].median())

            missing_cols_in_full = set(X_train.columns) - set(X_full_for_pred.columns)
            for c in missing_cols_in_full:
                X_full_for_pred[c] = 0
            X_full_for_pred = X_full_for_pred[X_train.columns]

            y_pred_all = modelo_rf.predict(X_full_for_pred)
            y_proba_all = modelo_rf.predict_proba(X_full_for_pred)

            df_predicciones_clientes = pd.DataFrame({
                'ID Cliente': clientes_ids_para_prediccion,
                'EsClienteAltoValor_pred': y_pred_all,
                'ProbBajoValor': y_proba_all[:, 0],
                'ProbAltoValor': y_proba_all[:, 1]
            })

            print("\nPredicciones generadas para todos los clientes y guardadas en 'df_predicciones_clientes'.")
            print(df_predicciones_clientes.head())

            # --- NUEVO: Separar y guardar clientes de alto y bajo valor ---
            clientes_alto_valor_ids = df_predicciones_clientes[df_predicciones_clientes['EsClienteAltoValor_pred'] == 1]['ID Cliente'].tolist()
            clientes_bajo_valor_ids = df_predicciones_clientes[df_predicciones_clientes['EsClienteAltoValor_pred'] == 0]['ID Cliente'].tolist()

            print(f"\nClientes de Alto Valor ({len(clientes_alto_valor_ids)}):")
            # Imprime solo los primeros 10 para no saturar la salida
            print(clientes_alto_valor_ids[:10])
            if len(clientes_alto_valor_ids) > 10:
                print(f"... y {len(clientes_alto_valor_ids) - 10} más.")

            print(f"\nClientes de Bajo Valor ({len(clientes_bajo_valor_ids)}):")
            # Imprime solo los primeros 10
            print(clientes_bajo_valor_ids[:10])
            if len(clientes_bajo_valor_ids) > 10:
                print(f"... y {len(clientes_bajo_valor_ids) - 10} más.")
            # ---------------------------------------------------

            # Evaluación (mantenemos la parte de test para verificar el rendimiento del modelo)
            print("\nRealizando predicciones en el conjunto de prueba...")
            y_pred_test = modelo_rf.predict(X_test)
            print("Predicciones completadas.")

            print("\n=== Evaluación del modelo ===")
            print("Precisión:", accuracy_score(y_test, y_pred_test))
            print("Este valor de precisión muestra qué tan bien el modelo predice la etiqueta 'Alto Valor'.")
            print("En un entorno real, valores de 80-95% son excelentes, indicando que el modelo ha aprendido bien los patrones.")
            print("\nReporte de clasificación:\n", classification_report(y_test, y_pred_test))
            print("El reporte de clasificación ofrece métricas más detalladas (precision, recall, f1-score) para cada clase (0: Bajo Valor, 1: Alto Valor).")
            print("Esto ayuda a entender qué tan bien el modelo identifica cada tipo de cliente.")
            print("============================")
        else:
            print("Error: El DataFrame X_ml (variables predictoras) está vacío o contiene columnas no numéricas después de la codificación.")
            if X_ml.empty: print("X_ml está vacío.")
            if not X_ml.select_dtypes(exclude=np.number).empty: print("Columnas no numéricas en X_ml:", X_ml.select_dtypes(exclude=np.number).columns.tolist())

        # Eliminar el objetivo temporal 'EsClienteAltoValor' de df_final para que no se use en el merge más adelante si no es deseado
        df_final.drop(columns=['EsClienteAltoValor'], errors='ignore', inplace=True)

    else:
        print("Error: La columna objetivo 'valor_monetario_total' no se encuentra en df_final para crear la columna objetivo.")
else:
    print("Error: df_final no existe o está vacío. Asegúrate de ejecutar las celdas anteriores que crean df_final.")



Columna 'EsClienteAltoValor' creada basada en la mediana del valor monetario.
Conteo de valores en la nueva columna objetivo:
EsClienteAltoValor
0    4164
1    4164
Name: count, dtype: int64

Preparando datos para predecir: 'EsClienteAltoValor'
Variables predictoras (X_ml): ['Tamaño de Cliente', 'recencia_dias', 'frecuencia_compras', 'std_venta_usd']

Entrenando el modelo RandomForestClassifier...
Entrenamiento completado.

Predicciones generadas para todos los clientes y guardadas en 'df_predicciones_clientes'.
   ID Cliente  EsClienteAltoValor_pred  ProbBajoValor  ProbAltoValor
0           1                        0            1.0            0.0
1          10                        1            0.0            1.0
2         100                        1            0.0            1.0
3        1001                        1            0.0            1.0
4        1002                        1            0.0            1.0

Clientes de Alto Valor (4159):
[10, 100, 1001, 1002, 1004, 1007, 1

In [13]:
import random
from datetime import datetime, timedelta
import pandas as pd
import numpy as np


# --- UNIÓN DE DATAFRAMES PARA df_clientes_completo ---
# Esta parte se ejecuta después de que el código de IA ha creado df_predicciones_clientes

if 'df_predicciones_clientes' in locals() and not df_predicciones_clientes.empty and \
   'df_final' in locals() and not df_final.empty:

    df_clientes_completo = pd.merge(df_predicciones_clientes, df_final, on='ID Cliente', how='left')

    # === SIMULACIÓN DE CARACTERÍSTICAS ADICIONALES PARA COCA-COLA ===
    if 'ID Cliente' in df_clientes_completo.columns and not 'Preferencia_Sabor' in df_clientes_completo.columns:
        num_clientes = len(df_clientes_completo)

        sabores = ['Dulce', 'Cítrico', 'Ligero', 'Intenso', 'Frutal', 'Neutro']
        momentos = ['Comidas', 'Mañana', 'Tarde', 'Noche', 'Deporte', 'Social', 'Descanso', 'Trabajo']
        sensibilidades = ['Alta', 'Media', 'Baja']

        df_clientes_completo['Preferencia_Sabor'] = np.random.choice(sabores, num_clientes)
        df_clientes_completo['Momento_Consumo_Preferido'] = np.random.choice(momentos, num_clientes)
        df_clientes_completo['Sensibilidad_Precio'] = np.random.choice(sensibilidades, num_clientes, p=[0.3, 0.4, 0.3])

        print("\nCaracterísticas adicionales simuladas para un enfoque más profundo en Coca-Cola.")

    # Simular una temporada actual para retos estacionales
    temporada_actual = "Verano" # ejemplo
    # ======================================================================

    print("\nDataFrame de clientes completo con predicciones y análisis detallado (df_clientes_completo):")
    print(df_clientes_completo.head())
    print("\nColumnas disponibles en df_clientes_completo (incluyendo las nuevas simuladas):")
    print(df_clientes_completo.columns.tolist())

else:
    print("Error: df_predicciones_clientes o df_final no existen o están vacíos. No se pudo crear df_clientes_completo.")

# --- CÓDIGO PARA LA FUNCIÓN GENERAR RETO (ADAPTADA AL ENTORNO COCA-COLA) ---
# Esta función genera UN ÚNICO reto personalizado por llamada.
def generar_reto_para_cliente(cliente_id, df_clientes_completo_param, temporada_actual="General"):
    cliente_data = df_clientes_completo_param[df_clientes_completo_param['ID Cliente'] == cliente_id]

    if cliente_data.empty:
        print(f"Error: Cliente {cliente_id} no encontrado en el DataFrame de clientes.")
        return None

    cliente_data = cliente_data.iloc[0]

    fecha_inicio_reto = datetime.now()
    fecha_fin_reto = fecha_inicio_reto + timedelta(days=random.randint(15, 30))

    reto = {
        "ID_Cliente": int(cliente_id),
        "Nombre_Reto": "",
        "Descripcion": "",
        "Puntos_Recompensa": 0,
        "Vigencia_Inicio": fecha_inicio_reto.strftime('%Y-%m-%d'),
        "Vigencia_Fin": fecha_fin_reto.strftime('%Y-%m-%d'),
        "Tipo_Reto": "",
        "Categoria_Enfocada": "General",
        "Producto_Sugerido": "Variedad Coca-Cola",
        "Mensaje_Emocional": ""
    }

    is_alto_valor = cliente_data['EsClienteAltoValor_pred'] if 'EsClienteAltoValor_pred' in cliente_data else 0
    prob_alto_valor = cliente_data['ProbAltoValor'] if 'ProbAltoValor' in cliente_data else 0.5

    categorias_ventas = {}
    for col in df_clientes_completo_param.columns:
        if col.startswith('Ventas_') and col in cliente_data:
            categoria_nombre = col.replace('Ventas_', '').replace('_', ' ')
            categorias_ventas[categoria_nombre] = cliente_data[col]

    categoria_mas_comprada = "Bebidas Refrescantes"
    categoria_menos_comprada = "Otras Bebidas"

    if categorias_ventas:
        categorias_con_ventas = {k: v for k, v in categorias_ventas.items() if v > 0}
        categorias_sin_ventas = [k for k, v in categorias_ventas.items() if v == 0]

        if categorias_con_ventas:
            categoria_mas_comprada = max(categorias_con_ventas, key=categorias_con_ventas.get)
            if categorias_sin_ventas:
                categoria_menos_comprada = random.choice(categorias_sin_ventas)
            else:
                categoria_menos_comprada = min(categorias_con_ventas, key=categorias_con_ventas.get)
        elif categorias_sin_ventas:
            categoria_menos_comprada = random.choice(categorias_sin_ventas)

    preferencia_sabor = cliente_data.get('Preferencia_Sabor', 'Dulce')
    momento_consumo = cliente_data.get('Momento_Consumo_Preferido', 'Comidas')
    sensibilidad_precio = cliente_data.get('Sensibilidad_Precio', 'Media')

    monto_gasto_promedio_x_compra = cliente_data.get('valor_monetario_total', 100) / cliente_data.get('frecuencia_compras', 1) if cliente_data.get('frecuencia_compras', 1) > 0 else 100

    reto_mensajes = {
        "Alto_Valor": {
            "Recompensa por Lealtad": {
                "Nombre": f"¡Tu Lealtad Refresca Más, Cliente Premium!",
                "Descripcion": lambda cat_mas: f"Disfruta aún más de lo que amas. Por tu lealtad, recibe puntos extra en tu próxima compra de {cat_mas}. ¡Celebra cada momento con Coca-Cola!",
                "Puntos": lambda venta_esp: max(75, min(250, int(venta_esp * 0.007))), # % más alto para VIP
                "Producto": "Tu Favorito",
                "Emocional": "¡Eres parte esencial de la familia Coca-Cola!"
            },
            "Explora Nueva Categoría": {
                "Nombre": lambda cat_menos: f"Expande tu Sabor: ¡Descubre {cat_menos}!",
                "Descripcion": lambda cat_menos, monto: f"Siempre hay algo nuevo para probar. Explora nuestra variedad de {cat_menos} y realiza una compra de ${monto:.2f} USD para ganar puntos. ¿Listo para la aventura de sabor?",
                "Puntos": lambda venta_esp: int(venta_esp * 0.025), # Incentivo fuerte para nueva categoría
                "Producto": lambda cat_menos: f"Nuevos sabores en {cat_menos}",
                "Emocional": "¡Despierta tu curiosidad y sorprende a tu paladar!"
            },
            "Compra de Mayor Volumen": {
                "Nombre": f"¡Maximiza tu Sabor, Multiplica tus Puntos!",
                "Descripcion": lambda monto: f"Para tus grandes momentos, grandes recompensas. Alcanza una compra de ${monto:.2f} USD este mes y dobla tus puntos. ¡Ideal para compartir!",
                "Puntos": lambda venta_esp: int(venta_esp * 0.02),
                "Producto": "Paquetes Familiares/Fiesta",
                "Emocional": "¡Que nada te falte para celebrar tus éxitos!"
            }
        },
        "Bajo_Valor": {
            "Primera Compra en Categoría": {
                "Nombre": lambda cat_menos: f"¡Tu Primera Chispa en {cat_menos} y Duplica tus Puntos!",
                "Descripcion": lambda cat_menos, monto: f"Un nuevo mundo de frescura te espera. Prueba nuestra categoría de {cat_menos} con una compra mínima de ${monto:.2f} USD y **duplica tus puntos de bienvenida**.",
                "Puntos": lambda venta_esp: int(venta_esp * 0.04), # DOBLE DE OFERTA
                "Producto": lambda cat_menos: f"Producto estrella de {cat_menos}",
                "Emocional": "¡Anímate a probar y te sorprenderás con el doble de recompensa!"
            },
            "Aumenta tu Frecuencia": {
                "Nombre": f"¡Más Veces, Más Refresco, DOBLE de Puntos!",
                "Descripcion": lambda freq: f"Queremos verte más seguido. Realiza {freq} compras este mes y **duplica los puntos** acumulados. ¡Cada momento con Coca-Cola es especial!",
                "Puntos": lambda venta_esp: int(venta_esp * 0.04), # DOBLE DE OFERTA
                "Producto": "Bebida Individual",
                "Emocional": "¡Haz de Coca-Cola parte de tu día a día y gana el doble!"
            },
            "Compra Mínima Garantizada": {
                "Nombre": f"¡Tu Momento Refrescante a un Paso y DOBLE Recompensa!",
                "Descripcion": lambda monto: f"Un pequeño gasto, una gran recompensa. Realiza una compra de al menos ${monto:.2f} USD este mes y recibe el **doble de puntos** extra para tus próximas recompensas.",
                "Puntos": lambda venta_esp: int(venta_esp * 0.04), # DOBLE DE OFERTA
                "Producto": "Snack y Bebida",
                "Emocional": "¡Refresca tus ideas, impulsa tu día con el doble de beneficios!"
            }
        }
    }

    # --- Selección de Reto y Personalización Fina ---
    # Aquí se elige el tipo de reto basado en si es alto o bajo valor
    if is_alto_valor == 1:
        grupo_reto = reto_mensajes["Alto_Valor"]
        tipo_reto_elegido = random.choice(list(grupo_reto.keys()))
        info_reto = grupo_reto[tipo_reto_elegido]
    else: # Si es cliente de Bajo Valor
        grupo_reto = reto_mensajes["Bajo_Valor"]
        tipo_reto_elegido = random.choice(list(grupo_reto.keys()))
        info_reto = grupo_reto[tipo_reto_elegido]

    reto["Tipo_Reto"] = tipo_reto_elegido

    venta_esperada_reto = 0 # Inicializar para asegurar que siempre tenga un valor

    # Cálculos específicos para cada tipo de reto
    if tipo_reto_elegido == "Recompensa por Lealtad":
        venta_esperada_reto = cliente_data.get('valor_monetario_total', 100)
        reto["Nombre_Reto"] = info_reto["Nombre"]
        reto["Descripcion"] = info_reto["Descripcion"](categoria_mas_comprada)
        reto["Puntos_Recompensa"] = info_reto["Puntos"](venta_esperada_reto)
        reto["Categoria_Enfocada"] = categoria_mas_comprada
        reto["Producto_Sugerido"] = info_reto["Producto"]
        reto["Mensaje_Emocional"] = info_reto["Emocional"]

    elif tipo_reto_elegido == "Explora Nueva Categoría":
        venta_esperada_reto = 50 # Monto fijo para explorar nueva categoría
        reto["Nombre_Reto"] = info_reto["Nombre"](categoria_menos_comprada)
        reto["Descripcion"] = info_reto["Descripcion"](categoria_menos_comprada, venta_esperada_reto)
        reto["Puntos_Recompensa"] = info_reto["Puntos"](venta_esperada_reto)
        reto["Categoria_Enfocada"] = categoria_menos_comprada
        reto["Producto_Sugerido"] = info_reto["Producto"](categoria_menos_comprada)
        reto["Mensaje_Emocional"] = info_reto["Emocional"]

    elif tipo_reto_elegido == "Compra de Mayor Volumen":
        target_aumento_porcentaje = random.uniform(0.1, 0.2)
        monto_objetivo = monto_gasto_promedio_x_compra * (1 + target_aumento_porcentaje)
        monto_objetivo = max(100, monto_objetivo) # Mínimo sensato
        venta_esperada_reto = monto_objetivo
        reto["Nombre_Reto"] = info_reto["Nombre"]
        reto["Descripcion"] = info_reto["Descripcion"](monto_objetivo)
        reto["Puntos_Recompensa"] = info_reto["Puntos"](venta_esperada_reto)
        reto["Categoria_Enfocada"] = "General"
        reto["Producto_Sugerido"] = info_reto["Producto"]
        reto["Mensaje_Emocional"] = info_reto["Emocional"]

    elif tipo_reto_elegido == "Primera Compra en Categoría":
        venta_esperada_reto = 30 # Monto fijo bajo para primera compra
        reto["Nombre_Reto"] = info_reto["Nombre"](categoria_menos_comprada)
        reto["Descripcion"] = info_reto["Descripcion"](categoria_menos_comprada, venta_esperada_reto)
        reto["Puntos_Recompensa"] = info_reto["Puntos"](venta_esperada_reto)
        reto["Categoria_Enfocada"] = categoria_menos_comprada
        reto["Producto_Sugerido"] = info_reto["Producto"](categoria_menos_comprada)
        reto["Mensaje_Emocional"] = info_reto["Emocional"]

    elif tipo_reto_elegido == "Aumenta tu Frecuencia":
        frecuencia_actual = cliente_data.get('frecuencia_compras', 1)
        target_frecuencia = frecuencia_actual + 1 if frecuencia_actual > 0 else 2 # Al menos 1 compra más, o 2 si no hay historial
        venta_esperada_reto = monto_gasto_promedio_x_compra * target_frecuencia
        venta_esperada_reto = max(50, venta_esperada_reto) # Mínimo sensato
        reto["Nombre_Reto"] = info_reto["Nombre"]
        reto["Descripcion"] = info_reto["Descripcion"](target_frecuencia)
        reto["Puntos_Recompensa"] = info_reto["Puntos"](venta_esperada_reto)
        reto["Categoria_Enfocada"] = "General"
        reto["Producto_Sugerido"] = info_reto["Producto"]
        reto["Mensaje_Emocional"] = info_reto["Emocional"]

    elif tipo_reto_elegido == "Compra Mínima Garantizada":
        monto_minimo = 75
        venta_esperada_reto = monto_minimo
        reto["Nombre_Reto"] = info_reto["Nombre"]
        reto["Descripcion"] = info_reto["Descripcion"](monto_minimo)
        reto["Puntos_Recompensa"] = info_reto["Puntos"](venta_esperada_reto)
        reto["Categoria_Enfocada"] = "General"
        reto["Producto_Sugerido"] = info_reto["Producto"]
        reto["Mensaje_Emocional"] = info_reto["Emocional"]

    # Considerar la sensibilidad al precio para clientes de bajo valor
    # Esta lógica se aplica adicionalmente *después* de los cálculos de puntos del reto_mensajes.
    # Si los puntos ya se duplicaron arriba, esta parte podría aumentarlos aún más.
    if is_alto_valor == 0 and sensibilidad_precio == 'Alta':
        if "Compra Mínima Garantizada" in reto["Tipo_Reto"] and reto["Puntos_Recompensa"] > 0:
            # Aquí ya se duplican los puntos en reto_mensajes, así que este 1.2 es un aumento adicional
            # Si solo quieres que el doble sea por la oferta base, podrías quitar este multiplicador.
            reto["Descripcion"] += " ¡Oferta especial por tiempo limitado!"
            reto["Puntos_Recompensa"] = int(reto["Puntos_Recompensa"] * 1.2) # 20% más puntos

    # Considerar la preferencia de sabor y momento de consumo para mensajes y productos
    if reto["Producto_Sugerido"] == "Tu Favorito":
        if preferencia_sabor == 'Cítrico':
            reto["Producto_Sugerido"] = "Sprite o Fanta"
        elif preferencia_sabor == 'Ligero':
            reto["Producto_Sugerido"] = "Coca-Cola Zero Azúcar"
        elif preferencia_sabor == 'Intenso':
            reto["Producto_Sugerido"] = "Coca-Cola Original"
        elif preferencia_sabor == 'Frutal':
            reto["Producto_Sugerido"] = "Jugos Del Valle o Fanta"
        else:
            reto["Producto_Sugerido"] = "Coca-Cola Original"

    if momento_consumo == "Deporte":
        reto["Descripcion"] += " ¡Ideal para recargar energías después de tu entrenamiento!"
        if reto["Producto_Sugerido"] == "Variedad Coca-Cola":
            reto["Producto_Sugerido"] = "Agua Powerade o Smartwater"
    elif momento_consumo == "Social":
        reto["Descripcion"] += " ¡Perfecto para compartir con amigos y familia!"
        if reto["Producto_Sugerido"] == "Variedad Coca-Cola":
            reto["Producto_Sugerido"] = "Coca-Cola en presentaciones grandes"
    elif momento_consumo == "Descanso":
        reto["Descripcion"] += " ¡Tu acompañante ideal para esos momentos de relax!"
        if reto["Producto_Sugerido"] == "Variedad Coca-Cola":
            reto["Producto_Sugerido"] = "Coca-Cola Sin Cafeína o Agua"
    elif momento_consumo == "Trabajo":
        reto["Descripcion"] += " ¡Para mantenerte enfocado y refrescado en tu jornada laboral!"
        if reto["Producto_Sugerido"] == "Variedad Coca-Cola":
            reto["Producto_Sugerido"] = "Coca-Cola Energy o Coca-Cola Original"

    if temporada_actual == "Verano" and reto["Categoria_Enfocada"] in ["Bebidas Refrescantes", "General"]:
        reto["Descripcion"] += " ¡Refresca tu verano con Coca-Cola!"
        reto["Nombre_Reto"] += " - Reto de Verano"
        if reto["Producto_Sugerido"] == "Variedad Coca-Cola":
             reto["Producto_Sugerido"] = "Coca-Cola Original Helada"
    elif temporada_actual == "Navidad" and reto["Categoria_Enfocada"] in ["Bebidas Refrescantes", "General"]:
        reto["Descripcion"] += " ¡La magia de la Navidad se vive con Coca-Cola!"
        reto["Nombre_Reto"] += " - Reto Navideño"
        if reto["Producto_Sugerido"] == "Variedad Coca-Cola":
             reto["Producto_Sugerido"] = "Coca-Cola Edición Navideña"

    reto["Puntos_Recompensa"] = max(10, reto["Puntos_Recompensa"])

    return reto

# --- CÓDIGO PARA PROBAR LA GENERACIÓN DE RETOS (MODIFICADO para doble reto en Bajo Valor) ---
if 'df_clientes_completo' in locals() and not df_clientes_completo.empty:
    if not df_clientes_completo['ID Cliente'].empty:
        # Número de clientes aleatorios que quieres ver en cada ejecución
        num_clientes_a_mostrar = 5

        # Seleccionar clientes aleatorios sin repetir
        clientes_ejemplo = df_clientes_completo['ID Cliente'].sample(
            min(num_clientes_a_mostrar, len(df_clientes_completo)),
            random_state=None
        ).tolist()

        print(f"\nGenerando retos de prueba para {len(clientes_ejemplo)} clientes aleatorios...")

        for cliente_id_test in clientes_ejemplo:
            print(f"\n--- Reto(s) para Cliente ID: {cliente_id_test} ---")

            # Obtener el tipo de cliente para decidir si generar un reto o dos
            cliente_data_temp = df_clientes_completo[df_clientes_completo['ID Cliente'] == cliente_id_test]
            if not cliente_data_temp.empty:
                is_alto_valor = cliente_data_temp.iloc[0]['EsClienteAltoValor_pred']
            else:
                is_alto_valor = 1 # Por defecto, si no se encuentra, asumir alto valor para generar 1 reto.

            if is_alto_valor == 0: # Cliente de Bajo Valor: Generar dos retos
                print("  Cliente de Bajo Valor detectado: Generando DOBLE de retos.")
                for i in range(1, 3): # Bucle para generar 2 retos
                    reto_generado = generar_reto_para_cliente(cliente_id_test, df_clientes_completo, temporada_actual="Verano")
                    if reto_generado:
                        print(f"\n  --- Reto {i} para Cliente {cliente_id_test} ---")
                        for k, v in reto_generado.items():
                            print(f"    {k}: {v}")
                    else:
                        print(f"  No se pudo generar el reto {i} para el cliente {cliente_id_test}.")
            else: # Cliente de Alto Valor: Generar un solo reto
                reto_generado = generar_reto_para_cliente(cliente_id_test, df_clientes_completo, temporada_actual="Verano")
                if reto_generado:
                    print(f"\n  --- Reto para Cliente {cliente_id_test} (Alto Valor) ---")
                    for k, v in reto_generado.items():
                        print(f"    {k}: {v}")
                else:
                    print(f"  No se pudo generar reto para el cliente {cliente_id_test}.")
    else:
        print("No hay IDs de cliente en df_clientes_completo para generar retos de ejemplo.")
else:
    print("No se pudo generar retos porque df_clientes_completo no está disponible.")



Características adicionales simuladas para un enfoque más profundo en Coca-Cola.

DataFrame de clientes completo con predicciones y análisis detallado (df_clientes_completo):
   ID Cliente  EsClienteAltoValor_pred  ProbBajoValor  ProbAltoValor  \
0           1                        0            1.0            0.0   
1          10                        1            0.0            1.0   
2         100                        1            0.0            1.0   
3        1001                        1            0.0            1.0   
4        1002                        1            0.0            1.0   

  Tamaño de Cliente  recencia_dias  frecuencia_compras  valor_monetario_total  \
0             MICRO            303                 115                900.210   
1            GRANDE            395                 376               5169.420   
2           MEDIANO              0                 616              13873.875   
3             MICRO              0                 345             

In [14]:
import joblib

# Guarda el modelo entrenado
joblib.dump(modelo_rf, "modelo_rf.pkl")

# Guarda los label encoders (para las variables categóricas)
joblib.dump(label_encoders_ml, "encoders.pkl")

# Guarda las columnas en el orden usado en el modelo
joblib.dump(X_train.columns.tolist(), "columnas_modelo.pkl")


['columnas_modelo.pkl']

In [15]:
from google.colab import files
files.download("modelo_rf.pkl")
files.download("encoders.pkl")
files.download("columnas_modelo.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>